# GT Educação

O GT Educação enviou uma série de formulários com indicadores desejados para o painel. A maioria desses indicadores estão disponíveis no Portal de Dados Abertos da Prefeitura de São Paulo (http://dados.prefeitura.sp.gov.br/).

Portanto, vamos carregar o módulo de downloads do portal de dados abertos.

In [1]:
from core.downloads import dadosabertos as da

# Número de alunos da Rede Municipal de Ensino

O primeiro indicador é o número de alunos da rede municipal, segmentado por nível educacional (Creche, Pré Escola, Ensino Fundamental I, Ensino Fundamental II, EJA I, EJA II, Ensino Médio, Ed. Prof.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/demanda-e-matriculas. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'demanda-e-matriculas'
resources = da.package_resources(pkg_name, '23')
resources

Sabendo o id do arquivo de dezembro de 2023, vamos carregar o recurso como um dataframe.

In [ ]:
resource_id = 'a63afccb-1a19-4438-96b1-7a13c4ccb586'
mat_23 = da.load_resource(resource_id, pandas_header=[0,1])
mat_23

O dataframe contem dados de matrículas, matrículas em processo e demanda não atendida. Como desejamos apenas matrículas, vamos remover as outras colunas.

In [ ]:
mat_23 = mat_23[['Distrito', 'Matrículas']]
mat_23

Agora, sabendo que temos apenas dados sobre matrículas, vamos ajustar as colunas.

In [ ]:
mat_23 = (mat_23
 .droplevel(0, axis='columns')
 .rename(columns={'Unnamed: 0_level_1': 'Distrito'}))

mat_23

As últimas linhas do dataframe são apenas dados de totalização e notas, então vamos excluílas também.

In [ ]:
mat_23 = mat_23.loc[mat_23['Distrito'].str.lower()!='total']
mat_23 = mat_23.loc[~mat_23.iloc[:,1].isna()]
mat_23

Finalmente, vamos "despivotar" a tabela, transformando o nome das colunas em uma nova coluna chamada "Nível educacional" e unificando as colunas de valor sob uma coluna chamada matrículas.

In [ ]:
mat_23 = mat_23.melt(
    id_vars='Distrito',
    var_name='Nível Educacional',
    value_name='Matrículas'
)
mat_23